In [42]:
import os
import json
import glob
import pandas as pd
import numpy as np
import hopsworks

# Option to display all columns during debugging
pd.set_option('display.max_columns', None)

In [43]:
# 1. Load Raw Files
paths = sorted(glob.glob("data/raw/tomtom_snapshots_*.jsonl"))
print(f" Found {len(paths)} files.")

rows = []
for p in paths:
    with open(p, "r") as f:
        for line in f:
            # Loading line by line is robust for JSONL
            rows.append(json.loads(line))

df_raw = pd.DataFrame(rows)

# 2. Function to extract only useful features and rename to snake_case
def extract_features(payload):
    if not isinstance(payload, dict): 
        return {}
    
    data = payload.get("flowSegmentData", {})
    if not data: 
        return {}
    
    return {
        "frc": data.get("frc"),
        "current_speed": data.get("currentSpeed"),
        "free_flow_speed": data.get("freeFlowSpeed"),
        "current_travel_time": data.get("currentTravelTime"),
        "free_flow_travel_time": data.get("freeFlowTravelTime"),
        "confidence": data.get("confidence"),
        "road_closure": data.get("roadClosure"),
        # Removed "coordinates", "version", and "source" as they are not needed for the ML model
    }

# Apply extraction
features_df = df_raw["payload"].apply(extract_features).apply(pd.Series)

# Join with essential base columns
# We drop "source", "version", and "payload" here to keep it clean
df = df_raw[["timestamp_utc", "point_id", "error"]].join(features_df)

# Filter out rows where extraction failed (error is not null)
df = df[df["error"].isna()].drop(columns=["error"])

print(f" Base dataframe created. Shape: {df.shape}")

 Found 17 files.
 Base dataframe created. Shape: (26068, 9)


In [44]:
# 1. Handle Timestamps (UTC & 10-minute buckets)
df["timestamp_utc"] = pd.to_datetime(df["timestamp_utc"], utc=True)
df["ts_10m"] = df["timestamp_utc"].dt.floor("10min")

# 2. Type Casting
# Ensure point_id is string for consistency
df["point_id"] = df["point_id"].astype(str)

# Cast numeric columns
num_cols = ["current_speed", "free_flow_speed", "current_travel_time", "free_flow_travel_time", "confidence"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# --- CRITICAL FIX FOR HOPSWORKS/AVRO ---
# Hopsworks Avro serializer often fails with Pandas nullable "boolean" (BooleanDtype).
# We fill NaNs with False and convert to standard numpy 'bool'.
df["road_closure"] = df["road_closure"].fillna(False).astype(bool)

# 3. Feature Engineering
# Calculate ratios and replace infinites with NaN
df["speed_ratio"] = df["current_speed"] / df["free_flow_speed"]
df["delay_seconds"] = df["current_travel_time"] - df["free_flow_travel_time"]

df["speed_ratio"] = df["speed_ratio"].replace([np.inf, -np.inf], np.nan)

# 4. Deduplication
# Sort by ID, Time, and Confidence (descending) to keep the highest quality record per bucket
df = df.sort_values(
    by=["point_id", "ts_10m", "confidence", "timestamp_utc"],
    ascending=[True, True, False, False]
)

# Drop duplicates based on the primary key (Segment ID + Time Bucket)
df = df.drop_duplicates(subset=["point_id", "ts_10m"], keep="first")

print(f" Dataframe ready for upload. Unique rows: {len(df)}")
print("Data Types:\n", df.dtypes) 
# Verify 'road_closure' is 'bool' (numpy) and NOT 'boolean' (pandas nullable)

 Dataframe ready for upload. Unique rows: 26018
Data Types:
 timestamp_utc            datetime64[ns, UTC]
point_id                              object
frc                                   object
current_speed                        float64
free_flow_speed                      float64
current_travel_time                  float64
free_flow_travel_time                float64
confidence                           float64
road_closure                            bool
ts_10m                   datetime64[ns, UTC]
speed_ratio                          float64
delay_seconds                        float64
dtype: object


In [45]:
# 1. Login to Hopsworks
project = hopsworks.login(
    host="eu-west.cloud.hopsworks.ai",
    project="London_traffic"
)
fs = project.get_feature_store()

# 2. Get or Create the Feature Group
# Using snake_case for the name is better practice
traffic_fg = fs.get_or_create_feature_group(
    name="traffic_flow_fg",
    version=1,
    primary_key=["point_id", "ts_10m"],
    event_time="ts_10m",
    description="Traffic flow data aggregated by 10min buckets",
)

# 3. Insert Data
# wait_for_job=True ensures the cell waits until ingestion finishes (good for production/debugging)
traffic_fg.insert(df)

2026-01-11 11:41:33,771 INFO: Closing external client and cleaning up certificates.
2026-01-11 11:41:33,784 INFO: Connection closed.
2026-01-11 11:41:33,786 INFO: Initializing external client
2026-01-11 11:41:33,786 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2026-01-11 11:41:34,955 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/3209


Uploading Dataframe: 100.00% |██████████| Rows 26018/26018 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: traffic_flow_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/3209/jobs/named/traffic_flow_fg_1_offline_fg_materialization/executions


(Job('traffic_flow_fg_1_offline_fg_materialization', 'SPARK'), None)

In [46]:
#df to csv
df.to_csv("data/processed/traffic_flow_data.csv", index=False)